In [50]:
import psycopg2
import numpy as np
import pandas as pd

In [51]:
# DB Connect
conn = psycopg2.connect(
    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port=5439,
    dbname='dev',
    user='awsuser',
    password='cremaoAdmin1234qwer!!'
)
cursor = conn.cursor()

### 1. 연별 합계 sum

In [52]:
# 유동인구
# 2020
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012' # 쿼리문
cursor.execute(query) # 실행
f_tot_2020 = cursor.fetchall() # 출력
f_tot_2020 = f_tot_2020[0][0] # 값 꺼내기

# 2021
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
f_tot_2021 = cursor.fetchall()
f_tot_2021 = f_tot_2021[0][0]

# 2022
query = 'select sum(f_tot) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
f_tot_2022 = cursor.fetchall()
f_tot_2022 = f_tot_2022[0][0]

In [53]:
# 거주인구
# 2020
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
pop_2020 = cursor.fetchall()
pop_2020 = pop_2020[0][0]

# 2021
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
pop_2021 = cursor.fetchall()
pop_2021 = pop_2021[0][0]


# 2022
query = 'select sum(pop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
pop_2022 = cursor.fetchall()
pop_2022 = pop_2022[0][0]

In [54]:
# 직장인구
# 2020
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202001 and 202012'
cursor.execute(query)
wpop_2020 = cursor.fetchall()
wpop_2020 = wpop_2020[0][0]

# 2021
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202101 and 202112'
cursor.execute(query)
wpop_2021 = cursor.fetchall()
wpop_2021 = wpop_2021[0][0]

# 2022
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym between 202201 and 202212'
cursor.execute(query)
wpop_2022 = cursor.fetchall()
wpop_2022 = wpop_2022[0][0]

In [ ]:
# 2020
query = 'select sum(wpop) from m1.kt_bldg_xy_info where base_ym = 202001'
cursor.execute(query)
wpop_2020 = cursor.fetchall()
wpop_2020 = wpop_2020[0][0]

### 2. 전년대비 증감율

In [ ]:
# 결과 데이터레임 생성
DF = f_tot['f_tot']

In [61]:
list_ = ['1','2','3'] 
DF = pd.DataFrame(list_)

In [64]:
DF

,0,f_tot,pop,wpop
0,1,1,1,1
1,2,1,1,1
2,3,1,1,1


In [58]:
# 전년 대비 증감율 
def perYearIncrease(df_prior, df_current, df, col): # df_prior=전기, df_current=당기, df=전기 데이터프레임, col=추가할 컬럼명
    if (df_prior/df_current)*100 >= 120: # 120% 이상: 1
        df[col] = 1
    elif (df_prior/df_current)*100 < 120: # 120% 미만: 0
        df[col] = 0

In [ ]:
# 전년 대비 증감율 
def perYearIncrease(df_prior, df_current, df, col): # df_prior=전기, df_current=당기, df=전기 데이터프레임, col=추가할 컬럼명
    
    df[col] = 0
    for i in range(len(df)):

        if (df_prior/df_current)*100 >= 120: # 120% 이상: 1
            df[col][i] = 1
        else: # 120% 미만: 0
            df[col][i] = 0

In [66]:
(f_tot_2020/f_tot_2021)*100

92.54235668331245

In [63]:
# 실행
# 유동인구
perYearIncrease(f_tot_2020, f_tot_2021, DF, 'f_tot') # 2021년도 증감율
perYearIncrease(f_tot_2021, f_tot_2022, DF, 'f_tot') # 2022년도 증감율

# 거주인구
perYearIncrease(pop_2020, pop_2021, DF, 'pop') # 2021년도 증감율
perYearIncrease(pop_2021, pop_2022, DF, 'pop') # 2022년도 증감율

# 직장인구
perYearIncrease(wpop_2020, wpop_2021, DF, 'wpop') # 2021년도 증감율
perYearIncrease(wpop_2021, wpop_2022, DF, 'wpop') # 2022년도 증감율

In [11]:
# 비율 파악 (0 대비 1의 비율)
# 유동인구
len(df[df['f_tot']==1]) / len(df[df['f_tot']==0])*100

# 거주인구
len(df[df['pop']==1]) / len(df[df['pop']==0])*100

# 직장인구
len(df[df['wpop']==1]) / len(df[df['wpop']==0])*100

,f_tot
